In [1]:
# importar librerias y funciones utilitarias
from helpers import *
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

### Activación de Item en base al codigo del Item

In [2]:
## lectura del archivo excel mandado a papelera 
item_activar = pd.read_excel('DATA/item_activar.xlsx',dtype={'CODIGO_ITEM': str})
print(item_activar)

       CODIGO_ITEM  GRUPO  CLASE  FAMILIA  ITEM               DESCRIPCION                     CODIGO + DESCRIPCION TIPO BIEN TIPO BIEN PATRIMONIAL  PAPELERA
0     392217770001     39     22     1777     1                  CHEMBALO                  392217770001 - CHEMBALO         B                     S  PAPELERA
1     392218920001     39     22     1892     1                    CITACA                    392218920001 - CITACA         B                     S  PAPELERA
2     675090000001     67     50     9000     1               SUBLIMADORA               675090000001 - SUBLIMADORA         B                     S  PAPELERA
3     392221220001     39     22     2122     1                    CLARIN                    392221220001 - CLARIN         B                     S  PAPELERA
4     532299520001     53     22     9952     1                  VIBRADOR                  532299520001 - VIBRADOR         B                     S  PAPELERA
...            ...    ...    ...      ...   ...           

In [3]:
id_item_map = {}
id_items = item_activar['CODIGO_ITEM'].unique().tolist()
print(id_items[1])

392218920001


In [4]:
conn, cursor = None, None
try:
    conn, cursor = connect_to_db('prod')
    for item in id_items:
        # Guardamos el ID en el mapa
        id_item_map[item] = buscar_item_por_numero_de_documento(item, cursor)
finally:
    if conn:
        conn.close()
        print("Conexión cerrada correctamente.")

119668
119669
230370
119670
190201
316057
316065
316063
316064
316128
316668
3579
3426
3439
3440
3441
3444
3446
3449
3452
3453
3454
3455
3456
3457
3458
3459
3460
3461
3462
3463
3464
3465
3466
3470
3471
3472
3474
3475
3477
3478
3481
3482
3483
3484
3485
3486
3487
3488
3489
3490
3500
3501
3502
3504
3506
3507
3508
3510
3511
3512
3513
3514
3515
3516
3518
3519
3522
3529
3530
3531
3532
3580
3540
3541
3542
3547
3549
3552
3553
3554
3558
3559
3560
3561
3562
3563
3567
3568
3569
3572
3573
3576
3577
3578
3582
3583
3586
3587
3588
3589
3590
3592
3593
3596
3597
3598
3599
3603
3604
3605
3606
3607
3608
3609
3613
3614
3615
3616
3617
14330
14331
14332
14333
14343
14347
14353
14354
14355
14356
14359
14365
14366
14367
14368
14371
14372
14374
14377
14428
14430
14434
14435
14444
14445
14447
14451
14452
14453
14457
14640
14548
14550
14551
14629
14642
14643
14661
14662
14663
14664
14673
14676
14692
14702
14703
14718
14719
14836
14845
14846
14849
14874
14875
14876
14877
14878
14885
14903
14910
14920
14921
14923


In [5]:
# Convertimos todos los valores a string y los unimos con una coma
resultado = ",".join(map(str, id_item_map.values()))

# Guardamos el texto directamente
with open('id_item.txt', 'w') as f:
    f.write(resultado)

In [6]:
item_activar['id_item'] = item_activar['CODIGO_ITEM'].map(id_item_map).fillna(0).astype(str)
item_activar.to_excel('item_activar.xlsx', index=False)

### Ejecutar creación de tablas grupo,clase,familia

In [7]:
conn,cursor = connect_to_db('local')
ejecutar_sql_con_cursor(cursor,'DATA\\creacion_tablas.sql')


📄 Leyendo el script: DATA\creacion_tablas.sql
🚀 Ejecutando script...
❌ Error al ejecutar el script 'DATA\creacion_tablas.sql': la relación «item_grupo» ya existe



False

### Agregar Grupo Item 

In [8]:
grupo = pd.read_csv('DATA/Catalogo-SIGA_MEF.csv', engine='python', on_bad_lines='warn',dtype=str)
grupo.index = grupo.index + 1
grupo.head()

,TIPO_BIEN,GRUPO_BIEN,NOMBRE_GRUPO,CLASE_BIEN,NOMBRE_CLASE,FAMILIA_BIEN,NOMBRE_FAMILIA,ITEM_BIEN,NOMBRE_ITEM,NOMBRE_UNIDAD_MEDIDA
1,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0050,ABONADORAS EN GENERAL,0001,ABONADORAS EN GENERAL,UNIDAD
2,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0240,AGITADOR MAGNÉTICO,0001,AGITADOR MAGNETICO,UNIDAD
3,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0335,ALIMENTADOR AUTOMÁTICO PARA PECES,0001,ALIMENTADOR AUTOMATICO PARA PECES,UNIDAD
4,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0431,ARADOS EN GENERAL,0001,ARADOS EN GENERAL,UNIDAD
5,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0478,ASTILLADORA FORESTAL,0001,ASTILLADORA FORESTAL,UNIDAD


In [9]:
unique_grupos_df = grupo[['GRUPO_BIEN', 'NOMBRE_GRUPO','TIPO_BIEN']].drop_duplicates().reset_index(drop=True) 
unique_grupos_df = unique_grupos_df.sort_values(by=['TIPO_BIEN','GRUPO_BIEN'],ascending=[True, True]).reset_index(drop=True)
unique_grupos_df.index = unique_grupos_df.index+2
unique_grupos_df.loc[1] = ['00', 'SIN GRUPO', ' SIN TIPO']
unique_grupos_df = unique_grupos_df.sort_index()
print(unique_grupos_df)

    GRUPO_BIEN                                       NOMBRE_GRUPO  TIPO_BIEN
1           00                                          SIN GRUPO   SIN TIPO
2           02                                          ABRASIVOS          B
3           03  ALAMBRES, BARRAS, PLANCHAS, PERFILES Y SIMILAR...          B
4           04                                AGRÍCOLA Y PESQUERO          B
5           05  AGROPECUARIA, GANADERÍA  Y JARDINERÍA  :   REP...          B
..         ...                                                ...        ...
110         86  SERVICIOS RELACIONADOS CON ASUNTOS LEGALES, JU...          S
111         87                     SERVICIOS PUBLICOS Y GENERALES          S
112         88                  SERVICIOS REGULADOS POR EL ESTADO          S
113         90                              TRANSPORTE Y TRASLADO          S
114         94  USO TEMPORAL DE BIENES MUEBLES,  INMUEBLES Y O...          S

[114 rows x 3 columns]


In [10]:
sql_insert_groups = []
for index, row in unique_grupos_df.iterrows():
    codigo_grupo = row['GRUPO_BIEN']
    nombre_grupo = row['NOMBRE_GRUPO']
    tipo_bien = row['TIPO_BIEN']

    # Escape single quotes in NOMBRE_FAMILIA to prevent SQL syntax errors
    nombre_grupos_escaped = nombre_grupo.replace("'", "''")

    # Construct the SQL INSERT statement
    sql_groups_statement = f" (DEFAULT, '{codigo_grupo}', '{nombre_grupos_escaped}', '{tipo_bien}')"

    sql_insert_groups.append(sql_groups_statement)

for statement in sql_insert_groups[:5]:
    print(statement)

 (DEFAULT, '00', 'SIN GRUPO', ' SIN TIPO')
 (DEFAULT, '02', 'ABRASIVOS', 'B')
 (DEFAULT, '03', 'ALAMBRES, BARRAS, PLANCHAS, PERFILES Y SIMILARES DE METAL', 'B')
 (DEFAULT, '04', 'AGRÍCOLA Y PESQUERO', 'B')
 (DEFAULT, '05', 'AGROPECUARIA, GANADERÍA  Y JARDINERÍA  :   REPUESTOS, ACCESORIOS Y MATERIALES', 'B')


In [11]:
output_file = 'insert_grupo.sql'

with open(output_file, 'w', encoding='utf-8') as f:
    f.write('INSERT INTO bytsscom_bytsig.item_grupo (id_grupo, codigo_grupo,descripcion_grupo,tipo) VALUES\n')
    
    # 2. El ciclo (escribe cada línea de la lista)
    for i, statement in enumerate(sql_insert_groups):
        f.write(statement)
        
        # Opcional: Agregar coma si no es el último elemento
        if i < len(sql_insert_groups) - 1:
            f.write(',\n')
        else:
            f.write(';\n') # Punto y coma al final de todo

print(f"Successfully exported {len(sql_insert_groups)} SQL INSERT statements to '{output_file}'")

Successfully exported 114 SQL INSERT statements to 'insert_grupo.sql'


In [12]:
## insertando en la base de datos 
ejecutar_sql_con_cursor(cursor,'insert_grupo.sql')

📄 Leyendo el script: insert_grupo.sql
🚀 Ejecutando script...
✅ Script insert_grupo.sql ejecutado en la transacción actual.


True

### Proceso para agregar Clase Item

In [13]:
clase = pd.read_csv('DATA/Catalogo-SIGA_MEF.csv', engine='python', on_bad_lines='warn',dtype=str)
clase.index = clase.index + 1
clase.head()

,TIPO_BIEN,GRUPO_BIEN,NOMBRE_GRUPO,CLASE_BIEN,NOMBRE_CLASE,FAMILIA_BIEN,NOMBRE_FAMILIA,ITEM_BIEN,NOMBRE_ITEM,NOMBRE_UNIDAD_MEDIDA
1,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0050,ABONADORAS EN GENERAL,0001,ABONADORAS EN GENERAL,UNIDAD
2,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0240,AGITADOR MAGNÉTICO,0001,AGITADOR MAGNETICO,UNIDAD
3,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0335,ALIMENTADOR AUTOMÁTICO PARA PECES,0001,ALIMENTADOR AUTOMATICO PARA PECES,UNIDAD
4,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0431,ARADOS EN GENERAL,0001,ARADOS EN GENERAL,UNIDAD
5,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0478,ASTILLADORA FORESTAL,0001,ASTILLADORA FORESTAL,UNIDAD


In [14]:
## Agregar los codigo de familias y de items 
unique_clase_df = grupo[['CLASE_BIEN', 'NOMBRE_CLASE','NOMBRE_GRUPO']].drop_duplicates().reset_index(drop=True) 
unique_clase_df = unique_clase_df.sort_values(by='CLASE_BIEN',ascending= True).reset_index(drop=True)
unique_clase_df.index = unique_clase_df.index+2
unique_clase_df.loc[1] = ['00','SIN CLASE','SIN GRUPO']
unique_clase_df = unique_clase_df.sort_index()
print(unique_clase_df)



    CLASE_BIEN                                   NOMBRE_CLASE                                       NOMBRE_GRUPO
1           00                                      SIN CLASE                                          SIN GRUPO
2           01               SERV.DE ORGANIZACIÓN DE EVENTOS                 DEPORTES, RECREACION Y ESPECTÁCULOS
3           01                          SERVICIOS DE EMBALAJE                              EMBALAJE Y ALMACENAJE
4           01                        SERVICIO POR ATENCIONES                         ATENCIONES Y CELEBRACIONES
5           01            SERVICIOS DE TRASLADO DE DOCUMENTOS                              TRANSPORTE Y TRASLADO
..         ...                                            ...                                                ...
623         99   TABACO PRODUCTOS PARA FUMADORES Y SUSTITUTOS                  ALIMENTOS Y BEBIDAS PARA PERSONAS
624         99                              VAJILLAS Y AFINES  COCINA, COMEDOR, CAFETERÍA  :   R

In [15]:
# 1. Creamos el mapa: Nombre del Grupo -> Índice del DataFrame de Grupos
mapping_id_grupo = unique_grupos_df.drop_duplicates('NOMBRE_GRUPO').reset_index().set_index('NOMBRE_GRUPO')['index']
unique_clase_df['ID_GRUPO'] = unique_clase_df['NOMBRE_GRUPO'].map(mapping_id_grupo)
unique_clase_df = unique_clase_df.sort_values(by=['ID_GRUPO','CLASE_BIEN'],ascending=[True, True]).reset_index(drop=True)

unique_clase_df['ID_GRUPO'] = unique_clase_df['ID_GRUPO'].fillna(1).astype(int)
unique_clase_df = unique_clase_df.sort_values(by='ID_GRUPO', ascending=True)
print(unique_clase_df.head())

  CLASE_BIEN                         NOMBRE_CLASE NOMBRE_GRUPO  ID_GRUPO
0         00                            SIN CLASE    SIN GRUPO         1
1         05         AMOLADERAS O PIEDRAS ESMERIL    ABRASIVOS         2
2         29  DISCOS DE DESBASTE O RECTIFICADORES    ABRASIVOS         2
3         35          ESCOBILLAS DE RED Y ALAMBRE    ABRASIVOS         2
4         55               LIJAS, PASTAS Y AFINES    ABRASIVOS         2


In [16]:
## Insertar las clases 
sql_insert_clase = []
for index, row in unique_clase_df.iterrows():
    codigo_clase = row['CLASE_BIEN']
    nombre_clase = row['NOMBRE_CLASE']
    id_grupo = row['ID_GRUPO']


    # Escape single quotes in NOMBRE_CLASE to prevent SQL syntax errors
    nombre_clases_escaped = nombre_clase.replace("'", "''")

    # Construct the SQL INSERT statement
    sql_clase_statement = f" (DEFAULT, '{codigo_clase}', '{nombre_clases_escaped}', {id_grupo})"

    sql_insert_clase.append(sql_clase_statement)

for statement in sql_insert_clase[:5]:
    print(statement)

 (DEFAULT, '00', 'SIN CLASE', 1)
 (DEFAULT, '05', 'AMOLADERAS O PIEDRAS ESMERIL', 2)
 (DEFAULT, '29', 'DISCOS DE DESBASTE O RECTIFICADORES', 2)
 (DEFAULT, '35', 'ESCOBILLAS DE RED Y ALAMBRE', 2)
 (DEFAULT, '55', 'LIJAS, PASTAS Y AFINES', 2)


In [17]:
output_file = 'insert_clases.sql'

with open(output_file, 'w', encoding='utf-8') as f:
    f.write('INSERT INTO bytsscom_bytsig.item_clase (id_clase, codigo_clase,nombre_clase,id_grupo) VALUES\n')
    
    # 2. El ciclo (escribe cada línea de la lista)
    for i, statement in enumerate(sql_insert_clase):
        f.write(statement)
        
        if i < len(sql_insert_clase) - 1:
            f.write(',\n')
        else:
            f.write(';\n') # Punto y coma al final de todo

    # f.write('\n-- Fin de la inserción masiva\n')

print(f"Successfully exported {len(sql_insert_clase)} SQL INSERT statements to '{output_file}'")

Successfully exported 627 SQL INSERT statements to 'insert_clases.sql'


In [18]:
## insertando en la base de datos 
ejecutar_sql_con_cursor(cursor,'insert_clases.sql')

📄 Leyendo el script: insert_clases.sql
🚀 Ejecutando script...
✅ Script insert_clases.sql ejecutado en la transacción actual.


True

### Proceso para agregar familias Item

In [19]:
## Obtener la informacion proporcionada por el siga 
data = pd.read_csv('DATA/Catalogo-SIGA_MEF.csv', engine='python', on_bad_lines='warn',dtype=str)

In [20]:
data.index = data.index + 1
data.head()

,TIPO_BIEN,GRUPO_BIEN,NOMBRE_GRUPO,CLASE_BIEN,NOMBRE_CLASE,FAMILIA_BIEN,NOMBRE_FAMILIA,ITEM_BIEN,NOMBRE_ITEM,NOMBRE_UNIDAD_MEDIDA
1,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0050,ABONADORAS EN GENERAL,0001,ABONADORAS EN GENERAL,UNIDAD
2,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0240,AGITADOR MAGNÉTICO,0001,AGITADOR MAGNETICO,UNIDAD
3,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0335,ALIMENTADOR AUTOMÁTICO PARA PECES,0001,ALIMENTADOR AUTOMATICO PARA PECES,UNIDAD
4,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0431,ARADOS EN GENERAL,0001,ARADOS EN GENERAL,UNIDAD
5,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0478,ASTILLADORA FORESTAL,0001,ASTILLADORA FORESTAL,UNIDAD


In [21]:
## Agregar los codigo de familias y de items 
data['SECUENCIA_FAMILIA'] = data['GRUPO_BIEN'] + data['CLASE_BIEN'] + data['FAMILIA_BIEN']
data['CODIGO_ITEM'] = data['GRUPO_BIEN'] + data['CLASE_BIEN'] + data['FAMILIA_BIEN'] + data['ITEM_BIEN']
data['CONCATENADO'] = data['GRUPO_BIEN'] + data['CLASE_BIEN'] + data['FAMILIA_BIEN'] + data['ITEM_BIEN']+' - ' + data['NOMBRE_ITEM'] 
data.head()

,TIPO_BIEN,GRUPO_BIEN,NOMBRE_GRUPO,CLASE_BIEN,NOMBRE_CLASE,FAMILIA_BIEN,NOMBRE_FAMILIA,ITEM_BIEN,NOMBRE_ITEM,NOMBRE_UNIDAD_MEDIDA,SECUENCIA_FAMILIA,CODIGO_ITEM,CONCATENADO
1,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0050,ABONADORAS EN GENERAL,0001,ABONADORAS EN GENERAL,UNIDAD,04220050,042200500001,042200500001 - ABONADORAS EN GENERAL
2,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0240,AGITADOR MAGNÉTICO,0001,AGITADOR MAGNETICO,UNIDAD,04220240,042202400001,042202400001 - AGITADOR MAGNETICO
3,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0335,ALIMENTADOR AUTOMÁTICO PARA PECES,0001,ALIMENTADOR AUTOMATICO PARA PECES,UNIDAD,04220335,042203350001,042203350001 - ALIMENTADOR AUTOMATICO PARA PECES
4,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0431,ARADOS EN GENERAL,0001,ARADOS EN GENERAL,UNIDAD,04220431,042204310001,042204310001 - ARADOS EN GENERAL
5,B,04,AGRÍCOLA Y PESQUERO,22,EQUIPO AGRÍCOLAS Y PESQUEROS,0478,ASTILLADORA FORESTAL,0001,ASTILLADORA FORESTAL,UNIDAD,04220478,042204780001,042204780001 - ASTILLADORA FORESTAL


In [22]:
mapping_id_clase = unique_clase_df.drop_duplicates('NOMBRE_CLASE').reset_index().set_index('NOMBRE_CLASE')['index']
unique_families_df = data[['FAMILIA_BIEN','SECUENCIA_FAMILIA', 'NOMBRE_FAMILIA','TIPO_BIEN','NOMBRE_CLASE']].drop_duplicates()
unique_families_df['ID_CLASE'] = unique_families_df['NOMBRE_CLASE'].map(mapping_id_clase)
unique_families_df = unique_families_df.sort_values(by='ID_CLASE', ascending=True)
unique_families_df = unique_families_df.reset_index(drop=True)
unique_families_df.index = unique_families_df.index + 1
print(unique_families_df)

      FAMILIA_BIEN SECUENCIA_FAMILIA                                    NOMBRE_FAMILIA TIPO_BIEN                                       NOMBRE_CLASE  ID_CLASE
1             0002          02050002                         PIEDRAS DE ESMERIL RECTAS         B                       AMOLADERAS O PIEDRAS ESMERIL         1
2             0005          02050005                   PIEDRAS DE ESMERIL CAPA MONTADA         B                       AMOLADERAS O PIEDRAS ESMERIL         1
3             0004          02050004                     PIEDRAS DE ESMERIL CAPA RECTA         B                       AMOLADERAS O PIEDRAS ESMERIL         1
4             0003          02050003                        PIEDRAS DE ESMERIL CONICAS         B                       AMOLADERAS O PIEDRAS ESMERIL         1
5             0001          02050001              PIEDRAS DE ESMERIL CIRCULAR/ REDONDA         B                       AMOLADERAS O PIEDRAS ESMERIL         1
...            ...               ...                

In [23]:
sql_insert_statements = []
sql_insert_statements.insert(1, " (DEFAULT, '0000', '00000000', 'SIN FAMILIA',1)")
for index, row in unique_families_df.iterrows():
    codigo_familia = row['FAMILIA_BIEN']
    secuencia_familia = row['SECUENCIA_FAMILIA']
    nombre_familia = row['NOMBRE_FAMILIA']
    id_clase = row['ID_CLASE']

    # Escape single quotes in NOMBRE_FAMILIA to prevent SQL syntax errors
    nombre_familia_escaped = nombre_familia.replace("'", "''")

    # Construct the SQL INSERT statement
    sql_statement = f" (DEFAULT, '{codigo_familia}','{secuencia_familia}', '{nombre_familia_escaped}',{id_clase})"

    sql_insert_statements.append(sql_statement)

for statement in sql_insert_statements[:5]:
    print(statement)



 (DEFAULT, '0000', '00000000', 'SIN FAMILIA',1)
 (DEFAULT, '0002','02050002', 'PIEDRAS DE ESMERIL RECTAS',1)
 (DEFAULT, '0005','02050005', 'PIEDRAS DE ESMERIL CAPA MONTADA',1)
 (DEFAULT, '0004','02050004', 'PIEDRAS DE ESMERIL CAPA RECTA',1)
 (DEFAULT, '0003','02050003', 'PIEDRAS DE ESMERIL CONICAS',1)


In [24]:
output_file = 'insert_familias.sql'

sql_query_update = """
UPDATE bytsscom_bytsig.item i
SET id_familia = COALESCE(sub.id_familia_encontrada, 1)
FROM (
    SELECT i2.cod_item, f.id_familia AS id_familia_encontrada
    FROM bytsscom_bytsig.item i2
    LEFT JOIN bytsscom_bytsig.item_familia f
      ON substr(i2.cod_item, 1, length(i2.cod_item) - 4) = f.secuencia_familia
) AS sub
WHERE i.cod_item = sub.cod_item;
"""

with open(output_file, 'w', encoding='utf-8') as f:
    f.write('INSERT INTO bytsscom_bytsig.item_familia (id_familia, codigo_familia,secuencia_familia,nombre_familia,id_clase) VALUES\n')
    
    # 2. El ciclo (escribe cada línea de la lista)
    for i, statement in enumerate(sql_insert_statements):
        # Si tu lista solo tiene los valores "(val1, val2)", 
        # podrías necesitar comas entre ellos:
        f.write(statement)
        
        # Opcional: Agregar coma si no es el último elemento
        if i < len(sql_insert_statements) - 1:
            f.write(',\n')
        else:
            f.write(';\n') # Punto y coma al final de todo

    f.write('\n-- Fin de la inserción masiva\n')
    f.write(sql_query_update) 

print(f"Successfully exported {len(sql_insert_statements)} SQL INSERT statements to '{output_file}'")

Successfully exported 10483 SQL INSERT statements to 'insert_familias.sql'


In [25]:
## insertando en la base de datos 
ejecutar_sql_con_cursor(cursor,'insert_familias.sql')

📄 Leyendo el script: insert_familias.sql
🚀 Ejecutando script...
✅ Script insert_familias.sql ejecutado en la transacción actual.


True